# Session 6: Multi-Agent Orchestration

**Duration**: 90 minutes  
**Difficulty**: Advanced

## Learning Objectives

- 🎯 Design multi-agent architectures
- 🎯 Implement router agents  
- 🎯 Build specialist agents
- 🎯 Orchestrate agent workflows
- 🎯 Handle agent-to-agent communication

## 📚 What You'll Build

**SupportGenie v0.6**: Multi-Agent System

Specialized agents:
- **Router Agent**: Classifies and routes queries
- **Support Agent**: General inquiries
- **Technical Agent**: Technical issues
- **Sales Agent**: Orders and billing
- **Escalation Agent**: Complex cases

## Part 0: Setup

In [ ]:
# Install required packages
!pip install openai python-dotenv -q

print("✅ Packages installed!")

In [ ]:
import os
import json
import random
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI

# Set up API key
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    from getpass import getpass
    if 'OPENAI_API_KEY' not in os.environ:
        os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')
    print("✅ API key loaded")

# Initialize client
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

print("\n🚀 Ready to build multi-agent systems!")

## Part 1: Why Multi-Agent Systems?

### Single Agent Limitations
- Tries to do everything
- Generic responses
- Hard to maintain
- Doesn't scale well

### Multi-Agent Benefits
- Specialized expertise
- Better accuracy
- Easier to maintain
- Scalable architecture

### Architecture

```
Orchestrator Agent
    ↓
┌───┴────┬─────────┬──────────┐
↓        ↓         ↓          ↓
Support  Technical Sales  Escalation
Agent    Agent     Agent  Agent
```

## Part 2: Router Agent

The router classifies queries and routes them to the right specialist.

In [ ]:
class RouterAgent:
    """Classifies queries and routes to specialists"""
    
    CLASSIFICATION_PROMPT = """Classify the customer query into ONE category:
    
    Categories:
    - "support": General inquiries, policies, how-to questions
    - "technical": Product issues, troubleshooting, bugs
    - "sales": Orders, billing, payments, pricing
    - "escalation": Complaints, refunds, urgent issues, angry customers
    
    Return ONLY the category name, nothing else.
    
    Query: {query}
    Category:"""
    
    def __init__(self):
        self.client = OpenAI()
    
    def classify(self, query: str) -> str:
        """Classify query and return agent type"""
        
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{
                "role": "user",
                "content": self.CLASSIFICATION_PROMPT.format(query=query)
            }],
            temperature=0
        )
        
        classification = response.choices[0].message.content.strip().lower()
        
        # Validate classification
        valid_types = ["support", "technical", "sales", "escalation"]
        if classification not in valid_types:
            return "support"  # Default fallback
        
        return classification

# Test router
router = RouterAgent()

test_queries = [
    "How do I reset my password?",
    "My laptop won't turn on",
    "I want to cancel my order",
    "This is unacceptable! I demand a refund!"
]

print("🔀 Router Agent Classification:\n")
for query in test_queries:
    classification = router.classify(query)
    print(f"Query: \"{query}\"")
    print(f"→ Route to: {classification.upper()} agent\n")

## Part 3: Specialist Agents

Each specialist has its own expertise and system message.

In [ ]:
class BaseAgent:
    """Base class for specialist agents"""
    
    def __init__(self, system_message: str):
        self.client = OpenAI()
        self.system_message = system_message
    
    def process(self, query: str, context: dict = None) -> dict:
        """Process query with agent's expertise"""
        
        # Build context string
        context_str = ""
        if context:
            context_str = f"\n\nContext: {json.dumps(context)}"
        
        messages = [
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": query + context_str}
        ]
        
        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages
        )
        
        return {
            "response": response.choices[0].message.content,
            "agent_type": self.__class__.__name__,
            "needs_escalation": self._check_escalation(response.choices[0].message.content)
        }
    
    def _check_escalation(self, response: str) -> bool:
        """Check if response indicates escalation needed"""
        escalation_phrases = [
            "escalate",
            "human agent",
            "supervisor",
            "cannot resolve"
        ]
        return any(phrase in response.lower() for phrase in escalation_phrases)

print("✅ BaseAgent class defined")

In [ ]:
class SupportAgent(BaseAgent):
    """Handles general support inquiries"""
    
    SYSTEM_MESSAGE = """You are a customer support specialist.
    
    Your expertise:
    - Policies and procedures
    - Account questions
    - General how-to guidance
    - Navigation assistance
    
    Guidelines:
    - Be friendly and patient
    - Provide clear, step-by-step instructions
    - Reference relevant policies
    - Escalate if issue is technical or requires refund
    """
    
    def __init__(self):
        super().__init__(self.SYSTEM_MESSAGE)


class TechnicalAgent(BaseAgent):
    """Handles technical issues"""
    
    SYSTEM_MESSAGE = """You are a technical support specialist.
    
    Your expertise:
    - Troubleshooting product issues
    - Software/hardware diagnostics
    - Technical configuration
    - Bug identification
    
    Guidelines:
    - Ask diagnostic questions
    - Provide step-by-step troubleshooting
    - Be technical but clear
    - Escalate if hardware replacement needed
    """
    
    def __init__(self):
        super().__init__(self.SYSTEM_MESSAGE)


class SalesAgent(BaseAgent):
    """Handles orders and billing"""
    
    SYSTEM_MESSAGE = """You are a sales support specialist.
    
    Your expertise:
    - Order management
    - Billing and payments
    - Pricing questions
    - Product recommendations
    
    Guidelines:
    - Be helpful and consultative
    - Upsell when appropriate
    - Process order changes
    - Escalate refund requests
    """
    
    def __init__(self):
        super().__init__(self.SYSTEM_MESSAGE)


class EscalationAgent(BaseAgent):
    """Handles escalations and complex cases"""
    
    SYSTEM_MESSAGE = """You are a senior support specialist.
    
    Your expertise:
    - Complex issues
    - Customer complaints
    - Refund processing
    - Policy exceptions
    
    Guidelines:
    - Be empathetic and apologetic
    - Take ownership of issues
    - Offer solutions proactively
    - Prioritize customer satisfaction
    """
    
    def __init__(self):
        super().__init__(self.SYSTEM_MESSAGE)

print("✅ All specialist agents defined")

In [ ]:
# Test specialist agents

support_agent = SupportAgent()
tech_agent = TechnicalAgent()
sales_agent = SalesAgent()

print("Testing Specialist Agents:\n")
print("="*60)

# Test SupportAgent
print("\n1. SUPPORT AGENT")
result = support_agent.process("How do I reset my password?")
print(f"Response: {result['response'][:200]}...")

print("\n" + "="*60)

# Test TechnicalAgent
print("\n2. TECHNICAL AGENT")
result = tech_agent.process("My laptop won't turn on")
print(f"Response: {result['response'][:200]}...")

print("\n" + "="*60)

# Test SalesAgent
print("\n3. SALES AGENT")
result = sales_agent.process("Can I upgrade my order?")
print(f"Response: {result['response'][:200]}...")

## Part 4: Orchestrator Pattern

The orchestrator coordinates all agents.

In [ ]:
class AgentOrchestrator:
    """Orchestrates multiple specialist agents"""
    
    def __init__(self):
        self.router = RouterAgent()
        self.agents = {
            "support": SupportAgent(),
            "technical": TechnicalAgent(),
            "sales": SalesAgent(),
            "escalation": EscalationAgent()
        }
    
    def handle_query(self, query: str, context: dict = None) -> dict:
        """Route query to appropriate agent"""
        
        # 1. Route to appropriate agent
        agent_type = self.router.classify(query)
        print(f"🔀 Routing to: {agent_type.upper()} agent")
        
        # 2. Get specialist agent
        agent = self.agents[agent_type]
        
        # 3. Execute with specialist
        response = agent.process(query, context)
        
        # 4. Check if escalation needed
        if response['needs_escalation'] and agent_type != "escalation":
            print("⬆️  Escalating to senior agent...")
            response = self.agents["escalation"].process(query, context)
        
        return response

print("✅ AgentOrchestrator class defined")

In [ ]:
# Test orchestrator

orchestrator = AgentOrchestrator()

test_scenarios = [
    "How do I track my order?",
    "My screen has a dead pixel",
    "I want to cancel my subscription",
    "This is ridiculous! I've been waiting for 2 weeks!"
]

print("🎭 Testing Multi-Agent Orchestration\n")
print("="*60)

for i, query in enumerate(test_scenarios, 1):
    print(f"\n{i}. Query: \"{query}\"")
    print("-" * 60)
    
    result = orchestrator.handle_query(query)
    
    print(f"\n✅ Response from {result['agent_type']}:")
    print(result['response'][:300] + "...")
    print("\n" + "="*60)

## Part 5: Parallel Execution

Run multiple agents simultaneously and synthesize results.

In [ ]:
class ParallelWorkflow:
    """Execute multiple agents in parallel"""
    
    def __init__(self, agents: list):
        self.agents = agents
    
    def execute_parallel(self, query: str) -> dict:
        """Run all agents in parallel and synthesize results"""
        
        print(f"🔄 Running {len(self.agents)} agents in parallel...\n")
        
        # Execute in parallel
        with ThreadPoolExecutor(max_workers=len(self.agents)) as executor:
            futures = [
                executor.submit(agent.process, query) 
                for agent in self.agents
            ]
            results = [f.result() for f in futures]
        
        # Synthesize results
        return self.synthesize(query, results)
    
    def synthesize(self, query: str, results: list) -> dict:
        """Combine multiple agent responses"""
        
        # Prepare synthesis prompt
        responses_text = "\n\n".join([
            f"Agent {i+1} ({r['agent_type']}): {r['response']}"
            for i, r in enumerate(results)
        ])
        
        synthesis_prompt = f"""Multiple agents provided responses to this query:
        
Query: {query}

Responses:
{responses_text}

Synthesize the best answer by combining insights from all agents.
Provide a single, comprehensive response."""
        
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": synthesis_prompt}]
        )
        
        return {
            "synthesized_response": response.choices[0].message.content,
            "individual_responses": results
        }

print("✅ ParallelWorkflow class defined")

In [ ]:
# Test parallel execution

# Use multiple agents for a complex query
parallel_agents = [
    SupportAgent(),
    TechnicalAgent(),
    SalesAgent()
]

workflow = ParallelWorkflow(parallel_agents)

query = "I received a defective laptop. What are my options?"

print(f"Query: {query}\n")
print("="*60)

result = workflow.execute_parallel(query)

print("\n📊 Individual Agent Responses:")
for i, resp in enumerate(result['individual_responses'], 1):
    print(f"\n{i}. {resp['agent_type']}:")
    print(resp['response'][:200] + "...")

print("\n" + "="*60)
print("\n✨ Synthesized Response:")
print(result['synthesized_response'])

## Part 6: SupportGenie v0.6 - Complete Multi-Agent System

In [ ]:
class SupportGenieV6:
    """
    SupportGenie Version 0.6
    Multi-agent customer support system
    """
    
    def __init__(self):
        self.setup_agents()
    
    def setup_agents(self):
        """Initialize all agents"""
        self.router = RouterAgent()
        self.support_agent = SupportAgent()
        self.tech_agent = TechnicalAgent()
        self.sales_agent = SalesAgent()
        self.escalation_agent = EscalationAgent()
    
    def handle_query(self, query: str, customer_profile: dict = None) -> dict:
        """Handle customer query with multi-agent routing"""
        
        # Classify query
        agent_type = self.router.classify(query)
        
        print(f"  🔀 Classified as: {agent_type.upper()}")
        
        # Route to specialist
        if agent_type == "support":
            response = self.support_agent.process(query, customer_profile)
        elif agent_type == "technical":
            response = self.tech_agent.process(query, customer_profile)
        elif agent_type == "sales":
            response = self.sales_agent.process(query, customer_profile)
        else:
            response = self.escalation_agent.process(query, customer_profile)
        
        # Auto-escalate if needed
        if response['needs_escalation'] and agent_type != "escalation":
            print(f"  ⬆️  Auto-escalating to senior agent")
            response = self.escalation_agent.process(query, customer_profile)
        
        return response

print("✅ SupportGenie v0.6 class defined")

In [ ]:
# Test SupportGenie v0.6

genie = SupportGenieV6()

customer_profile = {
    "name": "Sarah Johnson",
    "tier": "Gold",
    "total_orders": 25
}

test_queries = [
    "What's your return policy?",
    "My device keeps crashing when I open the app",
    "I need to change my billing address",
    "I've been trying to get help for 3 days! This is unacceptable!"
]

print("🤖 SupportGenie v0.6 - Multi-Agent System\n")
print("="*60)

for i, query in enumerate(test_queries, 1):
    print(f"\n{i}. Customer: \"{query}\"")
    print("-" * 60)
    
    result = genie.handle_query(query, customer_profile)
    
    print(f"\n  Agent: {result['agent_type']}")
    print(f"\n  Response: {result['response'][:250]}...")
    print("\n" + "="*60)

## Exercises

### Exercise 1: Add Agent Voting

Implement a system where multiple agents vote on the best response.

In [ ]:
# Your code here
class VotingSystem:
    """
    Multiple agents vote on best response
    """
    # TODO: Implement voting logic
    pass

### Exercise 2: Sequential Agent Chain

Create a chain where agents pass results to each other sequentially.

In [ ]:
# Your code here
class SequentialWorkflow:
    """
    Agents execute in sequence, passing results forward
    """
    # TODO: Implement sequential execution
    pass

### Exercise 3: Add Metrics Tracking

Track which agents are used most frequently and their success rates.

In [ ]:
# Your code here
class AgentMetrics:
    """
    Track agent usage and performance
    """
    # TODO: Implement metrics tracking
    pass

## 🎉 Session 6 Complete!

### What You Learned:

✅ Multi-agent systems scale better  
✅ Router agents enable specialization  
✅ Orchestrators coordinate workflows  
✅ Agents can run sequentially or parallel  
✅ Specialization improves accuracy  
✅ SupportGenie now has specialized expertise!

### Next Steps:

In **Session 7: Evaluation & Testing**, you'll learn:
- Creating test datasets
- Automated evaluation frameworks
- LLM-as-judge pattern
- A/B testing different approaches
- Monitoring and metrics

---

**Continue to**: [Session 7: Evaluation →](07_Evaluation.ipynb)